<a href="https://colab.research.google.com/github/nickpalladino/mtg-semantic-search/blob/main/mtg_mlm_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 0: Install necessary libraries
%pip install transformers datasets polars orjson torch accelerate scikit-learn numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

NameError: name 'df_full_data' is not defined

In [1]:
import polars as pl
import orjson
import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModel, # We'll use this to get embeddings for the evaluation task
)
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# --- 1. DATA PREPARATION (with a Train/Test Split) ---

print("Step 1: Preparing the dataset with a train/test split...")

# Load your full MTG card data
try:
    df_full_data = pl.read_parquet("mtg_data.parquet")
except Exception as e:
    print(f"Error loading 'mtg_data.parquet': {e}")
    exit()

# Convert card data to JSON strings
docs = []
for row in df_full_data.iter_rows(named=True):
    row_dict = {k: v for k, v in row.items() if v is not None and k != "scryfallId"}
    row_str = orjson.dumps(row_dict, option=orjson.OPT_INDENT_2).decode("utf-8")
    docs.append(row_str)

hf_dataset = Dataset.from_dict({"text": docs})

# Split the dataset: 95% for training, 5% for evaluation
dataset_split = hf_dataset.train_test_split(test_size=0.05, seed=42)
# The result is a DatasetDict: {'train': ..., 'test': ...}

print(f"Dataset split into {len(dataset_split['train'])} training and {len(dataset_split['test'])} evaluation examples.")

Step 1: Preparing the dataset with a train/test split...
Dataset split into 31085 training and 1637 evaluation examples.


In [3]:
# --- 2. TOKENIZATION (for both splits) ---

print("\nStep 2: Setting up model and tokenizer...")
model_path = "Alibaba-NLP/gte-modernbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model_for_training = AutoModelForMaskedLM.from_pretrained(model_path)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=512)

print("Tokenizing datasets...")
tokenized_datasets = dataset_split.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])



Step 2: Setting up model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

Tokenizing datasets...


Map (num_proc=4):   0%|          | 0/31085 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=4):   0%|          | 0/1637 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [5]:
# --- 3. TRAINING SETUP (with Evaluation) ---

print("\nStep 3: Setting up the trainer with evaluation...")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./gte-mtg-base-results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=500,
    # --- Key additions for evaluation ---
    eval_strategy="epoch", # Evaluate at the end of each epoch
    per_device_eval_batch_size=8,
)

trainer = Trainer(
    model=model_for_training,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"], # Provide the evaluation set
)


Step 3: Setting up the trainer with evaluation...


In [6]:
# --- 4. RUN TRAINING AND EVALUATION ---
print("\nStep 4: Starting self-supervised training...")
trainer.train()

# After training, the trainer automatically evaluates on the test set.
# You can also call it manually:
eval_results = trainer.evaluate()
print(f"\n--- Method A: Validation Loss Results ---")
print(f"MLM-Trained Model Perplexity: {np.exp(eval_results['eval_loss']):.2f}")
print("Note: A lower perplexity score is better.")

final_model_output_path = "./gte-mtg-base"
trainer.save_model(final_model_output_path)
tokenizer.save_pretrained(final_model_output_path)
print(f"Trained model saved to {final_model_output_path}")



Step 4: Starting self-supervised training...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: v4narchist (v4narchist-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W0707 19:44:45.206000 1108 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Epoch,Training Loss,Validation Loss
1,0.379600,0.372326



--- Method A: Validation Loss Results ---
MLM-Trained Model Perplexity: 1.48
Note: A lower perplexity score is better.
Trained model saved to ./gte-mtg-base


In [8]:
# --- 5. METHOD B: "ODD ONE OUT" TASK EVALUATION ---

print("\n\nStep 5: Evaluating with the 'Odd One Out' task...")

# Define a small, hand-curated test set for this task
odd_one_out_test_set = [
    {"group": ["Llanowar Elves", "Elvish Mystic", "Fyndhorn Elves"], "outlier": "Lightning Bolt"},
    {"group": ["Counterspell", "Mana Leak", "Memory Lapse"], "outlier": "Swords to Plowshares"},
    {"group": ["Dark Ritual", "Cabal Ritual", "Rite of Flame"], "outlier": "Brainstorm"},
    {"group": ["Wrath of God", "Damnation", "Day of Judgment"], "outlier": "Sol Ring"},
]

def get_embedding(text, model, tokenizer, device="cuda:0"):
    """Helper function to get a single embedding."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Take the [CLS] token's hidden state and normalize it
        embedding = torch.nn.functional.normalize(outputs.last_hidden_state[:, 0], p=2, dim=1)
    return embedding.cpu().numpy()

def evaluate_odd_one_out(model, tokenizer, test_set, card_lookup):
    """Calculates accuracy on the odd one out task."""
    correct_predictions = 0
    device = model.device

    for item in test_set:
        # Get the full JSON text for each card
        group_texts = [card_lookup[name] for name in item["group"]]
        outlier_text = card_lookup[item["outlier"]]

        # Get embeddings for all cards in the test case
        all_cards = item["group"] + [item["outlier"]]
        all_texts = group_texts + [outlier_text]

        embeddings = get_embedding(all_texts, model, tokenizer, device=device)

        # Calculate the centroid (average vector) of all embeddings
        centroid = np.mean(embeddings, axis=0)

        # Calculate the distance of each embedding from the centroid
        distances = [cosine_similarity([emb], [centroid])[0][0] for emb in embeddings]

        # The outlier should be the one with the lowest cosine similarity (greatest distance)
        predicted_outlier_index = np.argmin(distances)
        predicted_outlier_name = all_cards[predicted_outlier_index]

        if predicted_outlier_name == item["outlier"]:
            correct_predictions += 1

    return correct_predictions / len(test_set)

# Create the name -> JSON lookup map from the full dataset
card_name_to_json = {row['name']: orjson.dumps({k: v for k, v in row.items() if v is not None and k != "scryfallId"}, option=orjson.OPT_INDENT_2).decode("utf-8") for row in df_full_data.iter_rows(named=True)}

# Load the base and fine-tuned models for embedding generation
device = "cuda:0" if torch.cuda.is_available() else "cpu"
base_model = AutoModel.from_pretrained(model_path).to(device)
mlm_tuned_model = AutoModel.from_pretrained(final_model_output_path).to(device)

# Evaluate both models
base_model_accuracy = evaluate_odd_one_out(base_model, tokenizer, odd_one_out_test_set, card_name_to_json)
mlm_tuned_accuracy = evaluate_odd_one_out(mlm_tuned_model, tokenizer, odd_one_out_test_set, card_name_to_json)

print(f"\n--- Method B: 'Odd One Out' Results ---")
print(f"Base Model Accuracy: {base_model_accuracy:.2%}")
print(f"MLM-Trained Model Accuracy: {mlm_tuned_accuracy:.2%}")



Step 5: Evaluating with the 'Odd One Out' task...

--- Method B: 'Odd One Out' Results ---
Base Model Accuracy: 100.00%
MLM-Trained Model Accuracy: 75.00%


In [15]:
# --- Finding Similar Cards ---

print("Finding similar cards for 'Pact of Negation' within the 'free counterspells' list...")

card_name_to_find = "Pact of Negation"

# Manually provided list of "free counterspells" for comparison
# You can edit this list with the specific cards you want to compare against.
comparison_card_names = ['Daze', 'Disrupting Shoal', 'Fierce Guardianship', 'Flare of Denial', 'Foil', 'Force of Negation', 'Force of Will', 'Mental Misstep', 'Mindbreak Trap', 'Not of This World', 'Pact of Negation', 'Subtlety', 'Thwart']


if card_name_to_find not in card_name_to_json:
    print(f"Error: '{card_name_to_find}' not found in the dataset.")
elif not comparison_card_names:
     print("Error: The list of comparison cards is empty.")
else:
    # Get the embedding for the target card
    pact_of_negation_text = card_name_to_json[card_name_to_find]
    pact_of_negation_embedding_base = get_embedding(pact_of_negation_text, base_model, tokenizer, device=device)
    pact_of_negation_embedding_mlm = get_embedding(pact_of_negation_text, mlm_tuned_model, tokenizer, device=device)

    # Get embeddings for the comparison cards
    comparison_card_texts = [card_name_to_json[name] for name in comparison_card_names if name in card_name_to_json]

    # Ensure we have texts for all comparison names
    if len(comparison_card_texts) != len(comparison_card_names):
        print("Warning: Some comparison cards were not found in the dataset and will be skipped.")
        # Update comparison_card_names to only include those found in the dataset
        comparison_card_names = [name for name in comparison_card_names if name in card_name_to_json]


    print(f"Generating embeddings for {len(comparison_card_names)} comparison cards...")

    # Generate embeddings for the comparison cards in a batch
    comparison_embeddings_base = get_embedding(comparison_card_texts, base_model, tokenizer, device=device)
    comparison_embeddings_mlm = get_embedding(comparison_card_texts, mlm_tuned_model, tokenizer, device=device)


    # Calculate similarities and find top N
    def find_top_n_similar_in_list(target_embedding, comparison_embeddings, comparison_names, n=10):
        # Pass the target_embedding directly (it should already be a 2D array)
        similarities = cosine_similarity(target_embedding, comparison_embeddings)[0]

        # Combine names and similarities and sort
        नाम_sim = sorted(zip(comparison_names, similarities), key=lambda item: item[1], reverse=True)

        top_n_cards = []
        for name, similarity in  नाम_sim:
            if name != card_name_to_find:
                top_n_cards.append((name, similarity))
            if len(top_n_cards) == n:
                break
        return top_n_cards

    print("\nFinding similar cards for the base model...")
    # We want to find similar cards within the list, not necessarily the top 10 overall
    # Let's print all similarities within the list for clarity
    similarities_base = find_top_n_similar_in_list(pact_of_negation_embedding_base, comparison_embeddings_base, comparison_card_names, n=len(comparison_card_names))


    print("Finding similar cards for the MLM-tuned model...")
    similarities_mlm = find_top_n_similar_in_list(pact_of_negation_embedding_mlm, comparison_embeddings_mlm, comparison_card_names, n=len(comparison_card_names))

    # Display results
    print(f"\n--- Similar Cards to '{card_name_to_find}' within 'free counterspells' (Base Model) ---")
    for card, similarity in similarities_base:
        print(f"- {card} (Similarity: {similarity:.4f})")

    print(f"\n--- Similar Cards to '{card_name_to_find}' within 'free counterspells' (MLM-Tuned Model) ---")
    for card, similarity in similarities_mlm:
        print(f"- {card} (Similarity: {similarity:.4f})")

Finding similar cards for 'Pact of Negation' within the 'free counterspells' list...
Generating embeddings for 13 comparison cards...

Finding similar cards for the base model...
Finding similar cards for the MLM-tuned model...

--- Similar Cards to 'Pact of Negation' within 'free counterspells' (Base Model) ---
- Force of Negation (Similarity: 0.8776)
- Force of Will (Similarity: 0.8040)
- Mental Misstep (Similarity: 0.7946)
- Not of This World (Similarity: 0.7842)
- Flare of Denial (Similarity: 0.7828)
- Foil (Similarity: 0.7641)
- Thwart (Similarity: 0.7561)
- Fierce Guardianship (Similarity: 0.7552)
- Daze (Similarity: 0.7492)
- Disrupting Shoal (Similarity: 0.7438)
- Mindbreak Trap (Similarity: 0.7429)
- Subtlety (Similarity: 0.7156)

--- Similar Cards to 'Pact of Negation' within 'free counterspells' (MLM-Tuned Model) ---
- Force of Will (Similarity: 0.9848)
- Mental Misstep (Similarity: 0.9837)
- Force of Negation (Similarity: 0.9822)
- Flare of Denial (Similarity: 0.9804)
- Min

In [22]:
from tqdm import tqdm
import torch.nn.functional as F # Import F for normalization

dataloader = torch.utils.data.DataLoader(docs, batch_size=64,
                                         shuffle=False,
                                         pin_memory=True,
                                         pin_memory_device=device)

dataset_embeddings = []
# Use the MLM-tuned model to generate embeddings
model_for_embedding = mlm_tuned_model # Or use base_model if preferred

for batch in tqdm(dataloader, smoothing=0):
    tokenized_batch = tokenizer(
        batch, max_length=8192, padding=True, truncation=True, return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        # Use the selected model for embedding generation
        outputs = model_for_embedding(**tokenized_batch)
        embeddings = outputs.last_hidden_state[:, 0].detach().cpu()
    dataset_embeddings.append(embeddings)

dataset_embeddings = torch.cat(dataset_embeddings)
dataset_embeddings = F.normalize(dataset_embeddings, p=2, dim=1)
display(dataset_embeddings.size())

100%|██████████| 512/512 [11:08<00:00,  1.31s/it]


torch.Size([32722, 768])

In [24]:
df_2 = df_full_data.with_columns(embedding=dataset_embeddings.cpu().numpy()).sort("name")

df_2

name,scryfallId,manaCost,type,text,power,toughness,loyalty,rarities,sets,embedding
str,str,str,str,str,str,str,str,list[enum],list[enum],"array[f32, 768]"
"""""Ach! Hans, Run!""""","""84f2c8f5-8e11-4639-b7de-00e4a2…","""{2}{R}{R}{G}{G}""","""Enchantment""","""At the beginning of your upkee…",null,null,null,"[""rare""]","[""UNH""]","[0.000735, 0.03461, … 0.024184]"
"""""Brims"" Barone, Midway Mobster""","""f3ffb0ab-d7bf-4136-9aa7-0a4965…","""{3}{W}{B}""","""Legendary Creature — Human Rog…","""When ~ enters, put a +1/+1 cou…","""5""","""4""",null,"[""uncommon""]","[""UNF""]","[-0.000786, 0.041092, … 0.026244]"
"""""Lifetime"" Pass Holder""","""42293306-aaea-4542-8df4-813823…","""{B}""","""Creature — Zombie Guest""","""This creature enters tapped.\n…","""2""","""1""",null,"[""rare""]","[""UNF""]","[-0.003972, 0.034115, … 0.022818]"
"""""Name Sticker"" Goblin""","""fd1442b4-da59-4042-835f-143c8d…","""{2}{R}""","""Creature — Goblin Guest""","""When this creature enters from…","""2""","""2""",null,"[""common""]","[""UNF""]","[-0.011874, 0.036878, … 0.023963]"
"""""Rumors of My Death . . .""""","""cb3587b9-e727-4f37-b4d6-1baa73…","""{2}{B}""","""Enchantment""","""{3}{B}, Exile a permanent you …",null,null,null,"[""uncommon""]","[""UST""]","[-0.00926, 0.034443, … 0.013504]"
…,…,…,…,…,…,…,…,…,…,…
"""Éomer, King of Rohan""","""f2c11695-f22b-44d5-937c-2578f2…","""{3}{R}{W}""","""Legendary Creature — Human Nob…","""Double strike\nÉomer enters wi…","""2""","""2""",null,"[""rare""]","[""LTC""]","[-0.005365, 0.050943, … 0.024067]"
"""Éomer, Marshal of Rohan""","""0bd31ce9-9551-4efe-8bd2-b97d8e…","""{2}{R}{R}""","""Legendary Creature — Human Kni…","""Haste\nWhenever one or more ot…","""4""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]","[-0.01778, 0.047768, … 0.03575]"
"""Éowyn, Fearless Knight""","""35b6da81-dc24-45b3-b94c-0d6fa0…","""{2}{R}{W}""","""Legendary Creature — Human Kni…","""Haste\nWhen Éowyn enters, exil…","""3""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]","[-0.017023, 0.039276, … 0.026869]"


In [25]:
df_2.write_parquet("mtg_mlm_embeddings.parquet")

In [32]:
embeddings = df_2["embedding"].to_numpy(allow_copy=True)
embeddings.shape

(32722, 768)

In [26]:
def fast_dot_product(query, matrix, k=3):
    dot_products = query @ matrix.T

    idx = np.argpartition(dot_products, -k)[-k:]
    idx = idx[np.argsort(dot_products[idx])[::-1]]

    score = dot_products[idx]

    return idx, score

In [49]:
def get_similar_cards(card_name):
    query_embed = (
        df_2.filter(pl.col("name") == card_name)["embedding"].to_numpy(allow_copy=True)
    )[0]

    idx, score = fast_dot_product(query_embed, embeddings, k=19 + 1)
    print(score)
    return df_2[idx]

In [51]:
import polars as pl

# Set the maximum string length for display to a large value
pl.Config.set_fmt_str_lengths(500)
pl.Config.set_tbl_rows(20)

print("Polars display setting updated: Maximum string length for columns set to 500.")

Polars display setting updated: Maximum string length for columns set to 500.


In [53]:
similar = get_similar_cards("Pact of Negation")
similar.select(similar.columns[:5])

[1.0000002  0.98619163 0.98533106 0.9852638  0.9848413  0.9845923
 0.98374784 0.9837083  0.9836135  0.98338866 0.9833843  0.983305
 0.9832907  0.9830738  0.98303664 0.9829824  0.9829679  0.9829341
 0.98268574 0.9826347 ]


name,scryfallId,manaCost,type,text
str,str,str,str,str
"""Pact of Negation""","""cb91055f-215a-4243-835b-35de515a2ba5""","""{0}""","""Instant""","""Counter target spell.\nAt the beginning of your next upkeep, pay {3}{U}{U}. If you don't, you lose the game."""
"""Arcane Denial""","""8a5e5463-2451-48b7-a924-a81d2dd99671""","""{1}{U}""","""Instant""","""Counter target spell. Its controller may draw up to two cards at the beginning of the next turn's upkeep.\nYou draw a card at the beginning of the next turn's upkeep."""
"""Intervention Pact""","""bca127ef-349e-4380-875d-49c14fa03b18""","""{0}""","""Instant""","""The next time a source of your choice would deal damage to you this turn, prevent that damage. You gain life equal to the damage prevented this way.\nAt the beginning of your next upkeep, pay {1}{W}{W}. If you don't, you lose the game."""
"""Summoner's Pact""","""0d143917-bf8b-4418-9d78-b3d641314de8""","""{0}""","""Instant""","""Search your library for a green creature card, reveal it, put it into your hand, then shuffle.\nAt the beginning of your next upkeep, pay {2}{G}{G}. If you don't, you lose the game."""
"""Force of Will""","""83240eaa-edb4-4bd3-b193-29470cf46828""","""{3}{U}{U}""","""Instant""","""You may pay 1 life and exile a blue card from your hand rather than pay this spell's mana cost.\nCounter target spell."""
"""Slaughter Pact""","""81e00341-030f-433e-b22b-aca42e0a88d2""","""{0}""","""Instant""","""Destroy target nonblack creature.\nAt the beginning of your next upkeep, pay {2}{B}. If you don't, you lose the game."""
"""Mental Misstep""","""09113591-df5c-403d-982e-c4e8ba14c5cd""","""{U/P}""","""Instant""","""({U/P} can be paid with either {U} or 2 life.)\nCounter target spell with mana value 1."""
"""Undersimplify""","""3eaebdc1-7a20-45db-9d45-0238fc917496""","""{1}{U}""","""Instant""","""Choose target spell. If it's a creature spell, it perpetually gets -2/-0. Counter that spell unless its controller pays {2}."""
"""Code of Constraint""","""14648a6c-49ee-48d3-a83e-a68225d843bf""","""{2}{U}""","""Instant""","""Target creature gets -4/-0 until end of turn.\nDraw a card.\nAddendum — If you cast this spell during your main phase, tap that creature and it doesn't untap during its controller's next untap step."""
